In [1303]:
import pandas as pnd
import heapq
from collections import Counter

iris = pnd.read_csv('iris.csv')
#source: https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv

In [1304]:
import numpy as np

data_train = np.array(iris.iloc[:, 0:4])

data_train

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

In [1305]:
cluster_num = 3
linkage = "single"

data = data_train
data_num = len(data)
col_num = len(data[0])

label = []
dm = np.zeros(shape=(data_num,data_num))
sda = np.zeros(shape=(data_num,3))

In [1306]:
def distance(i1,i2):
    dist = 0
    for attr in range(col_num):
        dist += abs(data[i1][attr] - data[i2][attr])
    return dist

In [1307]:
def init():
    for i in range(data_num):
        label.append(i)

    for i1 in range(data_num):
        for i2 in range(data_num):
            dm[i1][i2] = distance(i1,i2)

In [1308]:
def process_shortest_path():
    for i in range(data_num):
        j = 0
        min = 99999
        sda[i][0] = min
        sda[i][1] = i
        sda[i][2] = j
        while j < data_num:
            if(dm[i][j] < min and dm[i][j] != 0):
                min = dm[i][j]
                sda[i][0] = min
                sda[i][1] = i
                sda[i][2] = j
            j += 1

In [1309]:
def single_linkage(id1,id2):
    for y in range(data_num):
        if(dm[id1][y] < dm[id2][y]):
            dm[id2][y] = dm[id1][y]
        else:
            dm[id1][y] = dm[id2][y]
    
    for x in range(data_num):
        if(dm[x][id1] < dm[x][id2]):
            dm[x][id2] = dm[x][id1]
        else:
            dm[x][id1] = dm[x][id2]

In [1310]:
def complete_linkage(id1,id2):
    for y in range(data_num):
        if(dm[id1][y] and dm[id2][y]):
            if(dm[id1][y] > dm[id2][y]):
                dm[id2][y] = dm[id1][y]
            else:
                dm[id1][y] = dm[id2][y]
    
    for x in range(data_num):
        if(dm[x][id1] and dm[x][id2]):
            if(dm[x][id1] > dm[x][id2]):
                dm[x][id2] = dm[x][id1]
            else:
                dm[x][id1] = dm[x][id2]

In [1311]:
def average_linkage(id1,id2):
    for y in range(data_num):
        if(dm[id1][y] and dm[id2][y]):
            average = dm[id2][y] + dm[id1][y] / 2
            dm[id2][y] = average
            dm[id1][y] = average

    for x in range(data_num):
        if(dm[x][id1] and dm[x][id2]):
            average = dm[x][id1] + dm[x][id2] / 2
            dm[x][id2] = average
            dm[x][id1] = average

In [1312]:
def update_distance_matrix(id1,id2):
    if linkage == "single":
        single_linkage(id1,id2)
    elif linkage == "complete":
        complete_linkage(id1,id2)
    else:
        average_linkage(id1,id2)

In [1313]:
def is_outlier_big():
    uniqueL = list(set(label))
    unsortedL = Counter(label)
    val = []

    for i in range(len(uniqueL)):
        val.append(unsortedL[uniqueL[i]])

    outlier = data_num - sum(heapq.nlargest(cluster_num, val))
    
    return (float(outlier)/data_num >= 0.03)

In [1314]:
def merging():
    process_shortest_path()
    
    min = sda[0][0]
    id1 = sda[0][1]
    id2 = sda[0][2]
    for i in range(data_num):
        if(sda[i][0] < min):
            min = sda[i][0]
            id1 = sda[i][1]
            id2 = sda[i][2]
    id1 = id1.astype(np.int64)
    id2 = id2.astype(np.int64)
    if(id1 < id2):
        label[:] = [id1 if x==id2 else x for x in label]
    else:
        label[:] = [id2 if x==id1 else x for x in label]
    dm[id1][id2] = 0
    dm[id2][id1] = 0
    
    update_distance_matrix(id1,id2)
            
    cn = len(list(set(label)))
    
    if cluster_num != cn and is_outlier_big():
        merging()

In [1315]:
def AgglomerativeClustering():
    init()
    merging()

In [1316]:
AgglomerativeClustering()

cluster = []
result = Counter(label).most_common(cluster_num)
for i in range(cluster_num):
    print "Cluster " + str(i) + ": " + str(result[i][1])
    cluster.append(result[i][0])

for i in range(data_num):
    if not label[i] in cluster:
        label[i] = -1
print "Labels:"
print label

Cluster 0: 97
Cluster 1: 48
Cluster 2: 1
Labels:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, -1, 50, 50, -1, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, -1, 50, 50, 50, 50, 50, 50, 50]
